In [2]:
import psycopg2 as ps
import pandas as pd
import Preprocessing
import time
import numpy as np
from collections import Counter
import fuzzywuzzy as fuz

## Getting first time data

In [205]:
## Connect to database

conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
cursor = conn.cursor()

q = '''SELECT * FROM TWEETS
        WHERE TWEET_CREATED >= '2016-01-01' 
        AND TWEET_CREATED < '2018-01-01'; '''
cursor.execute(q)
results = cursor.fetchall()
colnames = [desc[0] 
        for desc in cursor.description]
cursor.close()
conn.close()

df = pd.DataFrame(results, columns=colnames)

In [207]:
start = time.time()
df_clean = Preprocessing.clean_text(df, 'tweet_text')
end = time.time()
print(str(end-start), 'ms')

10.157874822616577 ms


In [208]:
df_clean.clean_text = df_clean.clean.copy()
df_clean = df_clean.drop('clean', axis=1)

In [209]:
# shift column 'Name' to first position
clean_text = df_clean.pop('clean_text') 
# insert column using insert(position,column_name,
# first_column) function
df_clean.insert(6, 'clean_text', clean_text)

In [210]:
df_clean.to_csv('tweets-2016-2017-clean.csv', encoding='utf-8')

## Using data csv

In [99]:
short_df = df_clean[['tweet_id', 'clean_text', 
    'isincident', 'isaccident', 'isobstacle', 'isdanger', 'incident_desc']].copy()

In [3]:
# short_df = pd.read_csv('tweets-2018-2020-clean.csv')
short_df = pd.read_csv('tweets-2018-2020-clean.csv')
short_df.drop('Unnamed: 0', axis=1, inplace=True)
## Fill NaN with None (if condition is false, then None)
short_df = short_df.where(pd.notnull(short_df), None)

In [4]:
short_df

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
0,1013816364220518402,precaucion calle 50 anegada via,None,None,None,None,None,None
1,951132392906870784,recorrido mañanitas,None,None,None,None,None,None
2,1010612149826342914,inundada calle 70 san fransisco,True,None,None,True,inundacion,None
3,1013816934142562304,inundada calle 74 san francisco precaucion ruta,True,None,None,True,inundacion,None
4,1022561878969671680,inundada avenida frente mercado marisco,True,None,None,True,inundacion,None
...,...,...,...,...,...,...,...,...
87101,1344759940305612820,5km h desplazan autos via transistmica altura ...,True,None,True,False,None,reporte velocidad
87102,1344773829290287105,recibe resumen noticias capsulas video canal y...,False,False,False,False,None,None
87103,1344780610594856967,celebremos viven casa forma virtual expresemos...,None,None,None,None,None,None
87104,1344810167276363781,registra accidente transito via centenario alt...,True,True,False,False,accidente,None


In [5]:
len(short_df)

87106

In [192]:
## Count values still not analyzed
len(short_df[short_df.isincident.isnull()])

12352

In [8]:
pd.set_option('display.max_columns', 500)
pd.set_option("max_colwidth", 200)
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 100)

In [197]:
len(short_df[extract])

93

In [199]:
short_df.loc[extract, 'isincident'] = True
short_df.loc[extract, 'isaccident'] = False
short_df.loc[extract, 'isobstacle'] = False
short_df.loc[extract, 'isdanger'] = True
short_df.loc[extract, 'incident_desc1'] = 'transporte'

In [203]:
## Check the true with the keyword
extract = short_df.clean_text.str.contains('chorrera provincia panama oeste') & short_df.isincident.isnull()
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
8953,991028130965020676,encuentran cuerpo flotando chorro chorrera provincia panama oeste presume taxista desaparecido informacion desarrollo,None,None,None,None,None,None
12748,1006144975711633408,camion articulado calle larga rio chongo chorrera provincia panama oeste via,None,None,None,None,None,None
15446,1017122370388144129,registra detras supermercado rey chorrera provincia panama oeste,None,None,None,None,None,None
24672,1054003645904232449,impactos bala asesinan sujeto sector altos san francisco altura minisuper mauricio chorrera provincia panama oeste via,None,None,None,None,None,None
24682,1054018702587424768,impactos bala asesinan sujeto sector altos san francisco altura minisuper mauricio chorrera provincia panama oeste,None,None,None,None,None,None
24740,1054115815484731392,sujetos armados dispararon hombre encontraba afuera discoteca madrugada avenida libertador chorrera provincia panama oeste herido 21 años murio hospital nicolas solano,None,None,None,None,None,None
31622,1077637476330758144,continua ola robos panama ocasion sujetos armados robaron local barriada guadalupe chorrera provincia panama oeste,None,None,None,None,None,None
35348,1091099727423328256,reportan persona amenaza lanzarse cuarto piso hospital nicolas solano chorrera provincia panama oeste desarrollo,None,None,None,None,None,None
35351,1091100991255134209,reportan persona amenaza lanzarse cuarto piso hospital nicolas solano chorrera provincia panama oeste,None,None,None,None,None,None
35352,1091102672650940417,reportan persona amenaza lanzarse cuarto piso hospital nicolas solano chorrera provincia panama oeste,None,None,None,None,None,None


In [9]:
counts = Counter(' '.join(short_df[short_df.isincident.isnull()].clean_text).split())
# short_df.isaccident.isnull() & short_df.clean_text.notnull()
freq = pd.DataFrame(counts.most_common(80), columns=['word', 'count'])
freq

,word,count
0,via,2426
1,panama,2302
2,altura,1216
3,calle,672
4,san,628
5,direccion,606
6,provincia,587
7,te,568
8,personas,537
9,chorrera,535


# NTLK

In [11]:
from nltk import everygrams, word_tokenize, FreqDist

In [200]:
df = short_df[short_df.isincident.isnull()]
df['combinations'] = df.clean_text.apply(lambda x: [' '.join(word) for word in everygrams(word_tokenize(x), 4, 4)])

<ipython-input-200-431253419b9f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['combinations'] = df.clean_text.apply(lambda x: [' '.join(word) for word in everygrams(word_tokenize(x), 4, 4)])


In [201]:
df1 = pd.DataFrame(list(FreqDist([''.join(y) for x in df['combinations'] for y in x]).items()), columns=['combination', 'freq'])

In [202]:
df1.sort_values('freq', ascending=False).head(20)

,combination,freq
6227,chorrera provincia panama oeste,49
600,avenida domingo diaz altura,36
6352,arraijan provincia panama oeste,31
5504,autopista arraijan chorrera altura,31
22504,mantiene corredor norte altura,25
23285,corredor norte altura tinajitas,24
586,direccion panama centro via,23
60092,ocurre calles lunes viernes,23
60091,enterate ocurre calles lunes,23
52767,altura tinajitas ambas direcciones,22


# Accidentes

In [276]:
len(short_df[extract])

230

In [79]:
counts = Counter(' '.join(short_df[extract].clean_text).split())
# short_df.isaccident.isnull() & short_df.clean_text.notnull()
freq = pd.DataFrame(counts.most_common(20), columns=['word', 'count'])
freq.head(5)

,word,count
0,via,216
1,interamericana,186
2,altura,186
3,direccion,65
4,provincia,60


In [178]:
short_df.loc[extract, 'isincident'] = True
short_df.loc[extract, 'isaccident'] = True
# short_df.loc[extract, 'isobstacle'] = True
# short_df.loc[extract, 'isdanger'] = True
short_df.loc[extract, 'incident_desc'] = 'incidente'

In [156]:
len(short_df[extract])

51

In [177]:
extract = short_df.clean_text.str.contains('perd.*control') & short_df.isincident.isnull()
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
656,951542935560048642,conductor vehiculo termina ciclovia perder control auto avenida ecuador precaucion ruta,None,None,None,None,None,None
10444,997552662584418305,conductor camion perdio control salio via altura sajalices chame,None,None,None,None,None,None
10549,997865258906738689,conductor bus perdio control termino residencia calle florida rio abajo,None,None,None,None,None,None
20241,1035910484871012354,conductor termino quebrada perder control vehiculo madrugada hato montaña conocio persona efectos alcohol,None,None,None,None,None,None
38109,1104422468666707970,persona murio conductor camion perdiera control vehiculo saliera via cayendo vacio bomberos estuvieron labores rescate ocupantes detalles,None,None,None,None,None,None
53936,1168265125377302528,conductor perdio control auto via centenario altura merca panama,None,None,None,None,None,None
54076,1168903729321521157,conductor vehiculo perdio control impacto local san antonio,None,None,None,None,None,None
56595,1178685151217356800,conductor unidad perdio control vehiculo entrada corredor norte altura terminal 2,None,None,None,None,None,None
62055,1198367687312904198,conductor perdio control vehiculo termino residencia sector valle urraca distrito san miguelito,None,None,None,None,None,None
65612,1216492959887216640,via tonosi provincia santos reportan conductor perdio control volco,None,None,None,None,None,None


In [336]:
## Specific rows
short_df.loc[753, 'isdanger'] = False

In [ ]:
## Save data into the same CSV (override)
short_df.to_csv('tweets-2018-2020-clean.csv', encoding='utf-8')

In [208]:
## Check the true with the keyword
extract = short_df.clean_text.str.contains('accidente[s]?') & short_df.isaccident==True
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
14,947883288630300673,registra accidente transito autopista arraijan chorrera altura bique direccion interior pais,True,True,False,False,accidente,None
15,947885261849071616,imagenes accidente transito autopista arraijan chorrera altura bique direccion interior pais,True,True,False,False,accidente,None
17,947891590172901383,trafico afectado accidente transito autopista arraijan chorrera altura bique direccion interior pais,True,True,False,False,"tranque, accidente",None
18,947896257024004097,registra accidente transito autopista arraijan chorrera altura bique direccion interior via,True,True,False,False,accidente,None
19,947897042386505728,accidente transito autopista arraijan chorrera altura bique direccion interior pais via,True,True,False,False,accidente,None
20,947903020901257216,mantiene trafico afectado accidente transito autopista arraijan chorrera altura bique direccion interior pais,True,True,False,False,"tranque, accidente",None
21,947903052320858112,aguaceros city panama cuidado calles suficientes accidentes muertos cuidado 1 50pm,True,True,None,None,accidente,None
22,947904579659816960,heridos deja accidente transito carretera costa altura villa luisa provincia colon,True,True,False,False,accidente,None
24,947912385658064898,accidente transito autopista arraijan chorrera altura bique direccion interior pais via noticias,True,True,False,False,accidente,None
25,947912884964687872,accidente transito carretera costa altura villa luisa provincia colon deja 5 heridos,True,True,False,False,accidente,None


In [15]:
short_df.loc[extract, 'isincident'] = False
short_df.loc[extract, 'isaccident'] = False
# short_df.loc[extract, 'isobstacle'] = False
# short_df.loc[extract, 'isdanger'] = False
# short_df.loc[extract, 'incident_desc'] = None

# Inspect

In [372]:
## Inspect a certain keyword in CSV
short_df[extract].to_csv('colision-transito.csv', encoding='utf-8')

In [381]:
revision = pd.read_csv('colision-transito.csv')
revision = revision.where(pd.notnull(revision), None)

In [399]:
len(revision)

394

In [400]:
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==1].tweet_id), 'isaccident'] = True
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==0].tweet_id), 'isaccident'] = False
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==1].tweet_id), 'isincident'] = True
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==1].tweet_id), 'incident_desc'] = 'colision'
short_df.loc[short_df.tweet_id.isin(revision.tweet_id), :]

# Peligros

In [407]:
short_df.loc[extract, 'isincident'] = True
short_df.loc[extract, 'isaccident'] = False
short_df.loc[extract, 'isdanger'] = True
short_df.loc[extract, 'isobstacle'] = True
short_df.loc[extract, 'incident_desc'] = 'metro'

In [406]:
len(short_df[extract])

616

In [80]:
extract = short_df.clean_text.str.contains('interamericana altura') & short_df.isincident.isnull()
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
2676,962061429154070528,despejado via interamericana altura espiga chorrera direccion interior pais momentos,None,None,None,None,None,None
2786,962382972602200070,transito fluido via interamericana altura rio hato,None,None,None,None,None,None
2813,962529844239069185,20km un desplazan autos via interamericana altura loma campana direccion interior pais,None,None,None,None,None,None
3620,967191927304212480,caballo soltura rio hato caballo suelto via interamericana altura rio hato provincia cocle direccion anton deja agarrar video,None,None,None,None,None,None
3916,968893501109948416,desprendio tren trasero bus via interamericana altura valle anton direccion veraguas,None,None,None,None,None,None
5401,975142009106325504,mula deja caer sacos cementos via interamericana altura loma campana direccion interior pais,None,None,None,None,None,None
5402,975144800121913345,mula deja caer sacos cementos via interamericana altura loma campana direccion interior pais dato,None,None,None,None,None,None
5403,975145209653813249,vistas mula deja caer sacos cementos via interamericana altura loma campana direccion interior pais,None,None,None,None,None,None
5404,975145295649615872,mula deja caer sacos cementos via interamericana altura loma campana direccion interior pais,None,None,None,None,None,None
5406,975145799293243392,mula deja caer sacos cementos via interamericana altura loma campana,None,None,None,None,None,None


# Obstáculos

In [270]:
# len(short_df[short_df.isincident.isnull()])
len(short_df[extract])

67

In [188]:
counts = Counter(' '.join(short_df[extract].clean_text).split())
# short_df.isaccident.isnull() & short_df.clean_text.notnull()
freq = pd.DataFrame(counts.most_common(30), columns=['word', 'count'])
freq

,word,count
0,precaucion,307
1,via,128
2,altura,123
3,panama,105
4,direccion,101
5,ruta,80
6,centro,65
7,avenida,41
8,corredor,35
9,autopista,28


In [190]:
short_df.loc[extract, 'isincident'] = True
short_df.loc[extract, 'isaccident'] = False
short_df.loc[extract, 'isobstacle'] = False
short_df.loc[extract, 'isdanger'] = True
short_df.loc[extract, 'incident_desc1'] = ''

In [191]:
len(short_df[extract])

317

In [187]:
extract = short_df.clean_text.str.contains('precaucion') & short_df.isincident.isnull()
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
0,1013816364220518402,precaucion calle 50 anegada via,None,None,None,None,None,None
114,948308001710100482,auto avenida martires altura asamblea nacional direccion panama oeste precaucion,None,None,None,None,None,None
201,948995781570256897,fuerte viento reportan puntos costa momentos precaucion area,None,None,None,None,None,None
394,950040747352477696,calles provincia colon amanecieron agua conducir precaucion,None,None,None,None,None,None
1121,953990407679102977,caballos soltura entrada aeropuerto tocumen acceso corredor sur precaucion ruta,None,None,None,None,None,None
1153,954097181761376256,registra salida corredor sur altura chanis precaucion ruta,None,None,None,None,None,None
1699,957253230798524416,registra triple colsion via centenario altura puente direccion panama centro precaucion,None,None,None,None,None,None
2426,961228199471575042,mula derriba semaforo avenida federico boyd altura cruce calle 50 precaucion ruta,None,None,None,None,None,None
2573,961728200874299392,registra coision via principal costa direccion salir chanis precaucion ruta,None,None,None,None,None,None
3254,965421257125761024,reportan ganado soltura milla 21 carretera conduce guabito bocas toro precaucion señores conductores,None,None,None,None,None,None


# Save data

In [204]:
## Save data into the same CSV (override)
short_df.to_csv('tweets-2018-2020-clean.csv', encoding='utf-8')

In [9]:
# for i in range(2014, 2021):
#     df_clean[(df_clean.tweet_created>='%i-01-01'%i) & (df_clean.tweet_created<'%i-01-01'%(i+1))].to_csv('tweets_%i.csv'%(i), encoding='utf-8')
df_clean.to_csv('tweets-2018-2020.csv', encoding='utf-8')